In [ ]:
!pip install -q transformers accelerate bitsandbytes einops

In [ ]:
from google.colab import drive
import pandas as pd

# Monta Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_path_llama = '/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_LLaMA_conv_turns.csv'
csv_path_minerva = '/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_detailed_4_optimized.csv'
csv_path_qwen = '/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_qwen(TEAR).csv'


df_llama = pd.read_csv(csv_path_llama)
df_minerva = pd.read_csv(csv_path_minerva)
df_qwen = pd.read_csv(csv_path_qwen)

# GEMINI

In [ ]:
from google.colab import userdata
import google.generativeai as genai
import os

try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    if not API_KEY:
        raise ValueError("Google API Key not found in Colab secrets.")
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring API key: {e}")
    exit() # Exit if API key setup fails


JUDGE_SYSTEM_INSTRUCTION = """
Act as an expert linguist specialized in Old Italian (13th–15th century Tuscan) and its translation into modern Italian.
You will be given two sentences:
- One in archaic Italian (Tuscany, 13th–15th century)
- One in modern Italian (a proposed translation)

Your job is to rate the quality of the modern Italian translation on a scale from 1 to 5.

## Scoring guidelines:
- **1**: The translation is incorrect or meaningless.
- **2**: The translation is very inaccurate or misleading. It is too literal and fails to preserve the original meaning.
- **3**: Understandable translation, but with significant issues in vocabulary, structure, or historical interpretation. The main idea is present but distorted.
- **4**: Good translation that preserves the meaning but contains awkward phrasing or slightly outdated structures for modern Italian.
- **5**: Excellent translation that is fluent, accurate, and faithful to both the meaning and tone of the original. Uses natural modern Italian.


## Format:
---
**Score**: <number from 1 to 5>
**Reason**: <short justification>
---
"""


model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    system_instruction=JUDGE_SYSTEM_INSTRUCTION
)


## LLaMa

In [ ]:
df=df_llama
response = df["LLaMA"]
reference = df["Sentence"]

In [ ]:
import pandas as pd
import time

def get_translation_score(archaic, modern):
    prompt = f"""
Archaic sentence: {archaic}
Modern Italian translation: {modern}
"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()

        # Extract score from the model response
        for line in text.splitlines():
            if line.strip().startswith("**Score**"):
                return int(line.split(":")[1].strip())
    except Exception as e:
        print(f"Error in evaluation: {e}")
    return None  # fallback

# Generate model response for every row
scores = []
for i, row in df.iterrows():
    archaic = row['Sentence']
    modern = row['LLaMA']
    score = get_translation_score(archaic, modern)
    print(f"[{i}] Score: {score}")
    scores.append(score)
    time.sleep(3)  # rate limit

df['gemini_score'] = scores

df.to_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/LLaMA_eval_gem.csv", index=False)

[0] Score: 4
[1] Score: 5
[2] Score: 4
[3] Score: 4
[4] Score: 4
[5] Score: 2
[6] Score: 3
[7] Score: 5
[8] Score: 5
[9] Score: 5
[10] Score: 3
[11] Score: 5
[12] Score: 5
[13] Score: 5
[14] Score: 3
[15] Score: 5
[16] Score: 5
[17] Score: 3
[18] Score: 4
[19] Score: 5
[20] Score: 5
[21] Score: 5
[22] Score: 5
[23] Score: 5
[24] Score: 4
[25] Score: 5
[26] Score: 5
[27] Score: 5
[28] Score: 5
[29] Score: 5
[30] Score: 5
[31] Score: 3
[32] Score: 3
[33] Score: 3
[34] Score: 5
[35] Score: 4
[36] Score: 5
[37] Score: 5
[38] Score: 4
[39] Score: 4
[40] Score: 2
[41] Score: 4
[42] Score: 5
[43] Score: 5
[44] Score: 4
[45] Score: 5
[46] Score: 4
[47] Score: 5
[48] Score: 4
[49] Score: 5
[50] Score: 5
[51] Score: 4
[52] Score: 5
[53] Score: 4
[54] Score: 3
[55] Score: 5
[56] Score: 5
[57] Score: 5
[58] Score: 4
[59] Score: 4
[60] Score: 5
[61] Score: 5
[62] Score: 4
[63] Score: 4
[64] Score: 4
[65] Score: 5
[66] Score: 5
[67] Score: 3
[68] Score: 4
[69] Score: 5
[70] Score: 5
[71] Score: 4
[7

In [ ]:
import pandas as pd
import json


df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/LLaMA_eval_gem.csv")


jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Gemini Score": int(row["gemini_score"]) if not pd.isna(row["gemini_score"]) else None
    })


out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-llama-gemini.jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")


## Minerva

In [ ]:
df=df_minerva

response = df["minerva"]
reference = df["Sentence"]

In [ ]:
import pandas as pd
import time

def get_translation_score(archaic, modern):
    prompt = f"""
Archaic sentence: {archaic}
Modern Italian translation: {modern}
"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()


        for line in text.splitlines():
            if line.strip().startswith("**Score**"):
                return int(line.split(":")[1].strip())
    except Exception as e:
        print(f"Error in evaluation: {e}")
    return None


scores = []
for i, row in df.iterrows():
    archaic = row['Sentence']
    modern = row['minerva']
    score = get_translation_score(archaic, modern)
    print(f"[{i}] Score: {score}")
    scores.append(score)
    time.sleep(3)


df['gemini_score'] = scores

df.to_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Minerva_eval_gem.csv", index=False)

[0] Score: 4
[1] Score: 5
[2] Score: 4
[3] Score: 4
[4] Score: 4
[5] Score: 2
[6] Score: 4
[7] Score: 3
[8] Score: 3
[9] Score: 5
[10] Score: 5
[11] Score: 5
[12] Score: 4
[13] Score: 1
[14] Score: 3
[15] Score: 5
[16] Score: 5
[17] Score: 3
[18] Score: 5
[19] Score: 5
[20] Score: 4
[21] Score: 3
[22] Score: 5
[23] Score: 5
[24] Score: 4
[25] Score: 3
[26] Score: 5
[27] Score: 4
[28] Score: 4
[29] Score: 4
[30] Score: 2
[31] Score: 5
[32] Score: 2
[33] Score: 4
[34] Score: 4
[35] Score: 3
[36] Score: 3
[37] Score: 4
[38] Score: 3
[39] Score: 4
[40] Score: 1
[41] Score: 3
[42] Score: 4
[43] Score: 4
[44] Score: 4
[45] Score: 3
[46] Score: 4
[47] Score: 4
[48] Score: 4
[49] Score: 3
[50] Score: 4
[51] Score: 3
[52] Score: 5
[53] Score: 2
[54] Score: 5
[55] Score: 4
[56] Score: 5
[57] Score: 5
[58] Score: 5
[59] Score: 3
[60] Score: 5
[61] Score: 5
[62] Score: 3
[63] Score: 3
[64] Score: 3
[65] Score: 5
[66] Score: 5
[67] Score: 5
[68] Score: 4
[69] Score: 3
[70] Score: 2
[71] Score: 3
[7

In [ ]:
import pandas as pd
import json


df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Minerva_eval_gem.csv")


jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Gemini Score": int(row["gemini_score"]) if not pd.isna(row["gemini_score"]) else None
    })


out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-minerva-gemini.jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

## Qwen

In [ ]:
df=df_qwen

response = df_qwen["modern_translation"]
reference = df_qwen["Sentence"]

In [ ]:
import pandas as pd
import time


def get_translation_score(archaic, modern):
    prompt = f"""
Archaic sentence: {archaic}
Modern Italian translation: {modern}
"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()

        for line in text.splitlines():
            if line.strip().startswith("**Score**"):
                return int(line.split(":")[1].strip())
    except Exception as e:
        print(f"Error in evaluation: {e}")
    return None


scores = []
for i, row in df.iterrows():
    archaic = row['Sentence']
    print(archaic)
    modern = row["modern_translation"]
    print(modern)
    score = get_translation_score(archaic, modern)
    print(f"[{i}] Score: {score}")
    scores.append(score)
    time.sleep(3)


df['gemini_score'] = scores

df.to_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Qwen_eval_gem.csv", index=False)

quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
Quella guerra era l'opera perché etc. Dall'altra parte, Aias era un guerriero greco e valoroso armato, dotato di grande abilità, ma non era ricco di grande senno.
[0] Score: 3
crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
Crudele, prende vendetta di tutte le colpe secondo la legge, e non perdonerà a nessun cavaliere che pecca.
[1] Score: 5
Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
Non era ornato da una forza di animo superiore Ponio Aufidiano, cavaliere italiano.
[2] Score: 3
Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
Se questo piace a tutti e se il tempo ha bisogno di avere Pompeio come cavaliere e non come amico, non considererò più le destinazioni.
[

In [ ]:
import pandas as pd
import json

df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Qwen_eval_gem.csv")

jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Gemini Score": int(row["gemini_score"]) if not pd.isna(row["gemini_score"]) else None
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-qwen-gemini.jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")


# PROMETHEUS

In [ ]:
from huggingface_hub import login
login()  #Use your read token!!!!!!!!!!!!

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Unbabel/M-Prometheus-7B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
rubric = """
[Translation Accuracy, Fluency, and Faithfulness]
Score 1: The translation is incorrect or meaningless.
Score 2: The translation is very inaccurate or misleading. It is too literal and fails to preserve the original meaning.
Score 3: Understandable translation, but with significant issues in vocabulary, structure, or historical interpretation. The main idea is present but distorted.
Score 4: Good translation that preserves the meaning but contains awkward phrasing or slightly outdated structures for modern Italian.
Score 5: Excellent translation that is fluent, accurate, and faithful to both the meaning and tone of the original. Uses natural modern Italian.
"""

def build_prompt(original, response, human):
    return f"""
###Task Description:
You need to evaluate the response to an instruction, which is a translation from archaic Italian to modern Italian.
A reference translation is also provided to help you better judge the quality of the response.
Please only output a score that is an integer between 1 and 5, according to the rubric below.
Do not generate any explanation or feedback.

###Instruction:
{original}

###Reference Answer:
{human}

###Response to Evaluate:
{response}

###Score Rubric:
{rubric}

###The Final Score to return: [RESULT]
"""


## LLaMA

In [ ]:
auto_df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_LLaMA_conv_turns.csv")
human_df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_human_translations.csv")

df = pd.merge(auto_df, human_df, on="Sentence")

In [ ]:
import re

def evaluate_with_prometheus(original, response, human):
    prompt = build_prompt(original, response, human)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=32)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    match = re.search(r"\[RESULT\]\s*(\d)", decoded)
    if match:
        score = int(match.group(1))
    else:
        score = None

    print("Original sentence:", original)
    print("Prometheus Score:", score)
    print("=========================")

    return score


from tqdm import tqdm
tqdm.pandas()

df["prometheus_score"] = df.progress_apply(
    lambda row: evaluate_with_prometheus(row["Sentence"], row["LLaMA"], row["Translation"]),
    axis=1
)

df.to_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/LLaMA_eval_prom.csv", index=False)

  2%|▏         | 2/96 [00:21<16:37, 10.61s/it]

Original sentence: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
Prometheus Score: 5


  3%|▎         | 3/96 [00:42<23:28, 15.15s/it]

Original sentence: crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
Prometheus Score: 4


  4%|▍         | 4/96 [01:04<26:46, 17.47s/it]

...

Original sentence: pregollo che lo liberasse di quella obbligazione, in che egli l' aveva lasciato ubbligato. El gentile uomo assentì, e liberollo, e fecene carta.
Prometheus Score: 4


100%|██████████| 96/96 [33:56<00:00, 21.29s/it]

Original sentence: L'oro verrà dall'Aquilone. Che figuriamo noi per l'Aquilone, se non il populo Gentile congelato dal freddo del peccato, il qual populo tenne sotto il giogo della sua tirannia
Prometheus Score: 2


100%|██████████| 96/96 [34:17<00:00, 21.43s/it]

Original sentence: sia in mezzo tra me e te: con noi non puo' tu già più lungamente dimorare, ch'io non lo sofferrò e non lo lascerò.
Prometheus Score: 3


In [ ]:
import pandas as pd
import json

df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/LLaMA_eval_prom.csv")

jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Prometheus Score": int(row["prometheus_score"]) if not pd.isna(row["prometheus_score"]) else None
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-llama-prometheus.jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

## Minerva

In [ ]:
auto_df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_detailed_4_optimized.csv")
human_df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_human_translations.csv")

df = pd.merge(auto_df, human_df, on="Sentence")

In [ ]:
import re

def evaluate_with_prometheus(original, response, human):
    prompt = build_prompt(original, response, human)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=32)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    match = re.search(r"\[RESULT\]\s*(\d)", decoded)
    if match:
        score = int(match.group(1))
    else:
        score = None

    print("Original sentence:", original)
    print("Prometheus Score:", score)
    print("=========================")

    return score


from tqdm import tqdm
tqdm.pandas()

df["prometheus_score"] = df.progress_apply(
    lambda row: evaluate_with_prometheus(row["Sentence"], row["minerva"], row["Translation"]),
    axis=1
)

df.to_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Minerva_eval_prom.csv", index=False)

  2%|▏         | 2/97 [34:53<27:37:41, 1046.96s/it]

Original sentence: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
Prometheus Score: 5


In [ ]:
import pandas as pd
import json

df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Minerva_eval_prom.csv")

jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Prometheus Score": int(row["prometheus_score"]) if not pd.isna(row["prometheus_score"]) else None
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-minerva-prometheus.jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

## Qwen

In [ ]:
auto_df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_qwen(TEAR).csv")
human_df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_human_translations.csv")

df = pd.merge(auto_df, human_df, on="Sentence")

In [ ]:
import re

def evaluate_with_prometheus(original, response, human):
    prompt = build_prompt(original, response, human)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=32)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    #search the response using regex
    match = re.search(r"\[RESULT\]\s*(\d)", decoded)
    if match:
        score = int(match.group(1))
    else:
        score = None

    print("Original sentence:", original)
    print("Prometheus Score:", score)
    print("=========================")

    return score


from tqdm import tqdm
tqdm.pandas()

df["prometheus_score"] = df.progress_apply(
    lambda row: evaluate_with_prometheus(row["Sentence"], row["modern_translation"], row["Translation"]),
    axis=1
)

df.to_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Qwen_eval_prom.csv", index=False)


  2%|▏         | 2/97 [00:25<19:54, 12.57s/it]

Original sentence: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
Prometheus Score: 3


  3%|▎         | 3/97 [00:46<25:09, 16.05s/it]

Original sentence: crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
Prometheus Score: 5


  4%|▍         | 4/97 [01:07<27:57, 18.03s/it]

...

Original sentence: pregollo che lo liberasse di quella obbligazione, in che egli l' aveva lasciato ubbligato. El gentile uomo assentì, e liberollo, e fecene carta.
Prometheus Score: 3


100%|██████████| 97/97 [34:08<00:00, 21.40s/it]

Original sentence: L'oro verrà dall'Aquilone. Che figuriamo noi per l'Aquilone, se non il populo Gentile congelato dal freddo del peccato, il qual populo tenne sotto il giogo della sua tirannia
Prometheus Score: 3


100%|██████████| 97/97 [34:29<00:00, 21.33s/it]

Original sentence: sia in mezzo tra me e te: con noi non puo' tu già più lungamente dimorare, ch'io non lo sofferrò e non lo lascerò.
Prometheus Score: 3


In [ ]:
import pandas as pd
import json

df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/Dataset evaluated/Qwen_eval_prom.csv.csv")

jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["Sentence"],
        "Prometheus Score": int(row["prometheus_score"]) if not pd.isna(row["prometheus_score"]) else None
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-qwen-prometheus.jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")
